In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
from ipywidgets import interact
import scipy.stats as stats
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors
import matplotlib.pyplot as plt

import warnings # the warning spam is pointless and annoying
warnings.simplefilter(action="ignore", category=FutureWarning)

Populating the interactive namespace from numpy and matplotlib


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [118]:
path_to_data="/content/drive/MyDrive/filtered_headlines.csv"
df=pd.read_csv(path_to_data, header=0 )
df

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn,Clean_Headline,Language,Processed_Headline,Topic 0,Topic 1,Topic 2,Topic 3
0,99248.0,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,0.000000,-0.053300,-1,-1,-1,Obama Lays Wreath at Arlington National Cemete...,en,obama lay wreath arlington nation cemeteri pre...,0.017137,0.016772,0.949220,0.016871
1,10423.0,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,0.208333,-0.156386,-1,-1,-1,"Tim Haywood, investment director business-unit...",en,tim haywood invest director busi unit head fix...,0.015689,0.724651,0.016840,0.242821
2,18828.0,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-0.425210,0.139754,-1,-1,-1,"Nouriel Roubini, NYU professor and chairman at...",en,nouriel roubini nyu professor chairman roubini...,0.526804,0.436595,0.018314,0.018287
3,27788.0,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,0.000000,0.026064,-1,-1,-1,Finland's economy expanded marginally in the t...,en,finland economi expand margin three month end ...,0.014203,0.957587,0.014114,0.014096
4,27789.0,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,0.000000,0.141084,-1,-1,-1,Tourism and public spending continued to boost...,en,tourism public spend continu boost economi jan...,0.014868,0.955129,0.015017,0.014987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86280,61851.0,Stocks rise as investors key in on US economy ...,The June employment report is viewed as a cruc...,MarketWatch,economy,2016-07-07 15:31:05,0.104284,0.044943,-1,3,5,The June employment report is viewed as a cruc...,en,june employ report view crucial gaug health ec...,0.018213,0.945057,0.018308,0.018422
86281,61865.0,Russian PM proposes to use conservative and to...,"In addition, establish stimulating economic po...",TASS,economy,2016-07-07 15:31:10,0.072194,0.000000,-1,0,1,"In addition, establish stimulating economic po...",en,addit establish stimul econom polici budget co...,0.131143,0.820171,0.024560,0.024126
86282,104793.0,Palestinian Government Uses Foreign Aid To Pay...,The Palestinian government spends nearly $ mil...,Daily Caller,palestine,2016-07-07 15:38:26,0.291667,-0.139754,5,1,0,The Palestinian government spends nearly $ mil...,en,palestinian govern spend near million annual s...,0.518430,0.444394,0.019046,0.018131
86283,104794.0,Palestine Youth Orchestra prepares for first U...,Palestine Youth Orchestra prepares for first U...,Ahram Online,palestine,2016-07-07 15:59:22,0.121534,0.092313,0,0,0,Palestine Youth Orchestra prepares for first U...,en,palestin youth orchestra prepar first uk tour ...,0.125082,0.016564,0.398295,0.460058


In [119]:
#df = pd.read_csv("/Users/nuzhatnazmulnishi/Desktop/Winter 2024/MATH 509/Project/20240306/filtered_headlines.csv", header=0)

In [249]:
df = df[["IDLink", "Topic", "Facebook", "GooglePlus", "LinkedIn", "Topic 0", "Topic 1", "Topic 2", "Topic 3"]]
# df = df.loc[(df["Facebook"] != -1) & (df["GooglePlus"] != -1) & (df["LinkedIn"] != -1)]
df = df.loc[(df["Facebook"] > 1) & (df["GooglePlus"] > 1) & (df["LinkedIn"] > 1)]

obama = tf.cast(df["Topic 0"], tf.float32)
econ = tf.cast(df["Topic 1"], tf.float32)
pal = tf.cast(df["Topic 2"], tf.float32)
ms = tf.cast(df["Topic 3"], tf.float32)

fb = tf.cast(df["Facebook"], tf.float32)
gp = tf.cast(df["GooglePlus"], tf.float32)
li = tf.cast(df["LinkedIn"], tf.float32)

In [250]:
df

,IDLink,Topic,Facebook,GooglePlus,LinkedIn,Topic 0,Topic 1,Topic 2,Topic 3
690,292.0,economy,27,2,22,0.007695,0.873518,0.007711,0.111076
696,476.0,microsoft,4,2,4,0.044200,0.043614,0.044327,0.867860
700,15.0,economy,127,2,55,0.018618,0.942860,0.019751,0.018770
701,16.0,economy,8,3,6,0.017003,0.949058,0.017073,0.016867
702,25.0,economy,31,7,3,0.017446,0.947793,0.017859,0.016903
...,...,...,...,...,...,...,...,...,...
86085,104694.0,obama,171,8,3,0.020711,0.020000,0.644022,0.315267
86123,104728.0,obama,78,2,4,0.021499,0.019759,0.936857,0.021884
86133,104724.0,obama,83,7,86,0.016639,0.016260,0.950164,0.016937
86163,104789.0,obama,6,2,3,0.078233,0.018432,0.884313,0.019022


In [251]:
print(df.shape)
mean_value = df.iloc[:, -4:].mean()
std_value = df.iloc[:, -4:].std()
print('Mean:',mean_value)
print('STD:', std_value)

mean_fb = df['Facebook'].mean()

print("Mean of the column FB:", mean_fb)

(14762, 9)
Mean: Topic 0    0.170898
Topic 1    0.239065
Topic 2    0.263155
Topic 3    0.326883
dtype: float64
STD: Topic 0    0.276186
Topic 1    0.318835
Topic 2    0.341834
Topic 3    0.381596
dtype: float64
Mean of the column FB: 436.54687711692185


## Generative Model
\begin{align*}
    Facebook_{i} &\sim \text{Normal}(\mu_{i}, \sigma) \\
    \mu_{i} &= \alpha + \beta_1* X_1 + \beta_2 * X_2 + \beta_3 * X_3 \\
    \alpha &\sim \chi^2 (1) \\
    \beta_1 & \sim \text{Normal}(0.203525, 0.29028) \\
    \beta_2 & \sim \text{Normal}(0.290288, 0.344120) \\
    \beta_3 & \sim \text{Normal}(0.258918, 0.338090) \\
    \sigma &\sim \text{Uniform}(0, 1000)
\end{align*}


In [252]:
np.random.seed(8927)

Nsteps = 500
N_burn_in = Nsteps
Nchains = 4

## FaceBook

In [253]:
#Obama = tf.math.log(tf.cast(df['Topic 0'],tf.float32))
#Econ = tf.math.log(tf.cast(df['Topic 1'],tf.float32))
#Palistine = tf.math.log(tf.cast(df['Topic 2'],tf.float32))

Obama = tf.cast(df['Topic 0'],tf.float32)
Econ = tf.cast(df['Topic 1'],tf.float32)
Palistine = tf.cast(df['Topic 2'],tf.float32)

FaceBook = tf.math.log(tf.math.log(tf.cast(df['Facebook'],tf.float32)))

In [254]:
#def custom_log(df):
 #   facebook_plus_one = df['Facebook'] + 1
 #   log_value = tf.math.log(tf.cast(facebook_plus_one, tf.float32))
 #   return log_value


In [255]:
def joint_log_prob(alpha, beta1, beta2, beta3, sigma):
    # Define priors
 #   alpha = tf.cast(alpha, tf.float64)
  #  beta1 = tf.cast(beta1, tf.float64)
   ##beta3 = tf.cast(beta3, tf.float64)
    #sigma = tf.cast(sigma, tf.float64)


 #   alpha_prior = tf.reduce_sum(tfp.distributions.Chi2(df=1, validate_args=False, allow_nan_stats=True).log_prob(alpha))
 #   beta1_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.203525, tf.float64), scale=tf.cast(0.298182, tf.float64)).log_prob(beta1))
 #   beta2_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.290288, tf.float64), scale=tf.cast(0.344120, tf.float64)).log_prob(beta2))
 #   beta3_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.258918, tf.float64), scale=tf.cast(0.338090, tf.float64)).log_prob(beta3))
 #   sigma_prior = tf.reduce_sum(tfp.distributions.Uniform(low=tf.cast(0.0, tf.float64), high=tf.cast(50.0, tf.float64)).log_prob(sigma))

    alpha_prior = tf.reduce_sum(tfp.distributions.Chi2(df=1, validate_args=False, allow_nan_stats=True).log_prob(alpha))
    beta1_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.203525, tf.float32), scale=tf.cast(0.298182, tf.float32)).log_prob(beta1))
    beta2_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.290288, tf.float32), scale=tf.cast(0.344120, tf.float32)).log_prob(beta2))
    beta3_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.258918, tf.float32), scale=tf.cast(0.338090, tf.float32)).log_prob(beta3))
    sigma_prior = tf.reduce_sum(tfp.distributions.Uniform(low=tf.cast(0.0, tf.float32), high=tf.cast(1000.0, tf.float32)).log_prob(sigma))

    mu = alpha + beta1_prior * Obama + beta2_prior  * Econ + beta3_prior  * Palistine

    _density_fb= tfd.Normal(loc=mu, scale=sigma).log_prob(FaceBook )
    #_density_fb= tfd.Exponential(1/mu).log_prob(FaceBook)
    density_fb = tf.reduce_sum(_density_fb, axis=0)

    # Sum of log probabilities
    return density_fb + alpha_prior + beta1_prior + beta2_prior + beta3_prior + sigma



In [256]:
# Define bijectors for transformations
bijectors = [
    tfp.bijectors.Square(),
    tfp.bijectors.Identity(),
    tfp.bijectors.Identity(),
    tfp.bijectors.Identity(),
    tfp.bijectors.SoftClip(low=0.0,high=1000.0)
]

# Initial state for the sampler
initial_state = [
    tf.zeros([], dtype=tf.float32, name='init_alpha'),
    tf.zeros([], dtype=tf.float32, name='init_beta1'),
    tf.zeros([], dtype=tf.float32, name='init_beta2'),
    tf.zeros([], dtype=tf.float32, name='init_beta3'),
    tf.ones([], dtype=tf.float32, name='init_sigma')
]

In [257]:
@tf.function()
def sample_chain(initial_state):
    theta = tfp.mcmc.sample_chain(
        #num_results=Nsteps,
        num_results=2000,
        num_burnin_steps=N_burn_in,
        # num_burnin_steps=1000,
        current_state=initial_state,
        kernel=tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.TransformedTransitionKernel(

                # NUTS instead of HMC
                inner_kernel=tfp.mcmc.NoUTurnSampler(
                    target_log_prob_fn=joint_log_prob,
                    max_tree_depth=8,
                    max_energy_diff=1000,

                    step_size=tf.cast(0.01, tf.float32)),    # STEPSIZE TUNE

                bijector= bijectors ),

            num_adaptation_steps=600),
        trace_fn=None)
    return theta


In [258]:
alpha = chisquare(1)
beta1 = normal(0.203525,0.298182)
beta2 = normal(0.290288,0.344120)
beta3 = normal(0.258918,0.338090)
sigma = uniform(0.0, 1000.0)

theta = sample_chain([alpha, beta1, beta2, beta3, sigma])
print(theta)

[<tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([1.3762288, 1.3855942, 1.3855942, ..., 1.3112103, 1.3112103,
       1.3112103], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.3582383 , 0.36343026, 0.36343026, ..., 0.23960839, 0.23960839,
       0.23960839], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.14917022, 0.1586061 , 0.1586061 , ..., 0.05955377, 0.05955377,
       0.05955377], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.24303472, 0.24410751, 0.24410751, ..., 0.26909247, 0.26909247,
       0.26909247], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([8.544373  , 8.49353   , 8.49353   , ..., 0.48083496, 0.48083496,
       0.48083496], dtype=float32)>]


In [259]:
# Extract samples
posterior = [theta[0].numpy(), theta[1].numpy(), theta[2].numpy(), theta[3].numpy(), theta[4].numpy()]

# Create a DataFrame
posterior_df = pd.DataFrame(posterior).T  # Transpose the DataFrame

# Set column names
posterior_df.columns = ['alpha', 'beta1', 'beta2', 'beta3', 'sigma']
print(posterior_df)

         alpha     beta1     beta2     beta3     sigma
0     1.376229  0.358238  0.149170  0.243035  8.544373
1     1.385594  0.363430  0.158606  0.244108  8.493530
2     1.385594  0.363430  0.158606  0.244108  8.493530
3     1.386643  0.358336  0.155572  0.246155  8.441284
4     1.382280  0.365867  0.150077  0.233318  8.393311
...        ...       ...       ...       ...       ...
1995  1.311210  0.239608  0.059554  0.269092  0.480835
1996  1.311210  0.239608  0.059554  0.269092  0.480835
1997  1.311210  0.239608  0.059554  0.269092  0.480835
1998  1.311210  0.239608  0.059554  0.269092  0.480835
1999  1.311210  0.239608  0.059554  0.269092  0.480835

[2000 rows x 5 columns]


## LinkedIn

In [260]:
Obama = tf.cast(df['Topic 0'],tf.float32)
Econ = tf.cast(df['Topic 1'],tf.float32)
Palistine = tf.cast(df['Topic 2'],tf.float32)

LinkedIn = tf.math.log(tf.math.log(tf.cast(df['LinkedIn'],tf.float32)))

In [191]:
#def custom_log(df):
 #   li_plus_one = df['LinkedIn'] + 1
 #   log_value = tf.math.log(tf.cast(li_plus_one, tf.float32))
 #   return log_value

In [261]:
def joint_log_prob(alpha, beta1, beta2, beta3, sigma):
    # Define priors

    alpha_prior = tf.reduce_sum(tfp.distributions.Chi2(df=1, validate_args=False, allow_nan_stats=True).log_prob(alpha))
    beta1_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.203525, tf.float32), scale=tf.cast(0.298182, tf.float32)).log_prob(beta1))
    beta2_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.290288, tf.float32), scale=tf.cast(0.344120, tf.float32)).log_prob(beta2))
    beta3_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.258918, tf.float32), scale=tf.cast(0.338090, tf.float32)).log_prob(beta3))
    sigma_prior = tf.reduce_sum(tfp.distributions.Uniform(low=tf.cast(0.0, tf.float32), high=tf.cast(1000.0, tf.float32)).log_prob(sigma))

    mu = alpha + beta1_prior * Obama + beta2_prior  * Econ + beta3_prior  * Palistine

    _density_li = tfd.Normal(loc=mu, scale=sigma).log_prob(LinkedIn)
    density_li = tf.reduce_sum(_density_li, axis=0)

    # Sum of log probabilities
    return density_li + alpha_prior + beta1_prior + beta2_prior + beta3_prior + sigma_prior


In [262]:
# Define bijectors for transformations
bijectors = [
    tfp.bijectors.Square(),
    tfp.bijectors.Identity(),
    tfp.bijectors.Identity(),
    tfp.bijectors.Identity(),
    tfp.bijectors.SoftClip(low=0.0,high=1000.0)
]

# Initial state for the sampler
initial_state = [
    tf.zeros([], dtype=tf.float32, name='init_alpha'),
    tf.zeros([], dtype=tf.float32, name='init_beta1'),
    tf.zeros([], dtype=tf.float32, name='init_beta2'),
    tf.zeros([], dtype=tf.float32, name='init_beta3'),
    tf.ones([], dtype=tf.float32, name='init_sigma')
]

In [263]:
@tf.function()
def sample_chain(initial_state):
    theta = tfp.mcmc.sample_chain(
        #num_results=Nsteps,
        num_results=2000,
        num_burnin_steps=N_burn_in,
        # num_burnin_steps=1000,
        current_state=initial_state,
        kernel=tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.TransformedTransitionKernel(

                # NUTS instead of HMC
                inner_kernel=tfp.mcmc.NoUTurnSampler(
                    target_log_prob_fn=joint_log_prob,
                    max_tree_depth=8,
                    max_energy_diff=1000,

                    step_size=tf.cast(0.01, tf.float32)),    # STEPSIZE TUNE

                bijector= bijectors ),

            num_adaptation_steps=600),
        trace_fn=None)
    return theta


In [264]:
alpha = chisquare(1)
beta1 = normal(0.203525,0.298182)
beta2 = normal(0.290288,0.344120)
beta3 = normal(0.258918,0.338090)
sigma = uniform(0.0, 1000.0)

theta = sample_chain([alpha, beta1, beta2, beta3, sigma])
print(theta)

[<tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.36632207, 0.36632207, 0.36632207, ..., 0.78047776, 0.78047776,
       0.78047776], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.10878567, 0.10878567, 0.10878567, ..., 0.0199116 , 0.0199116 ,
       0.0199116 ], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.35461757, 0.35461757, 0.35461757, ..., 0.21438769, 0.21438769,
       0.21438769], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.49085498, 0.49085498, 0.49085498, ..., 0.48562297, 0.48562297,
       0.48562297], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([80.877686, 80.877686, 80.877686, ..., 17.11499 , 17.11499 ,
       17.11499 ], dtype=float32)>]


In [265]:
# Extract samples
posterior = [theta[0].numpy(), theta[1].numpy(), theta[2].numpy(), theta[3].numpy(), theta[4].numpy()]

# Create a DataFrame
posterior_df = pd.DataFrame(posterior).T  # Transpose the DataFrame

# Set column names
posterior_df.columns = ['alpha', 'beta1', 'beta2', 'beta3', 'sigma']
print(posterior_df)

         alpha     beta1     beta2     beta3      sigma
0     0.366322  0.108786  0.354618  0.490855  80.877686
1     0.366322  0.108786  0.354618  0.490855  80.877686
2     0.366322  0.108786  0.354618  0.490855  80.877686
3     0.463957  0.193201  0.386834  0.451279  80.265442
4     0.364582  0.159202  0.319872  0.377345  79.622131
...        ...       ...       ...       ...        ...
1995  0.780478  0.019912  0.214388  0.485623  17.114990
1996  0.780478  0.019912  0.214388  0.485623  17.114990
1997  0.780478  0.019912  0.214388  0.485623  17.114990
1998  0.780478  0.019912  0.214388  0.485623  17.114990
1999  0.780478  0.019912  0.214388  0.485623  17.114990

[2000 rows x 5 columns]


## GooglePlus

In [266]:
#Obama = tf.math.log(tf.cast(df['Topic 0'],tf.float32))
#Econ = tf.math.log(tf.cast(df['Topic 1'],tf.float32))
#Palistine = tf.math.log(tf.cast(df['Topic 2'],tf.float32))

Obama = tf.cast(df['Topic 0'],tf.float32)
Econ = tf.cast(df['Topic 1'],tf.float32)
Palistine = tf.cast(df['Topic 2'],tf.float32)

GooglePlus = tf.math.log(tf.math.log(tf.cast(df['GooglePlus'],tf.float32)))

In [199]:
#def custom_log(df):
 #   gplus_plus_one = df['GooglePlus'] + 1
 #   log_value = tf.math.log(tf.cast(gplus_plus_one, tf.float32))
 #   return log_value

In [267]:
def joint_log_prob(alpha, beta1, beta2, beta3, sigma):
    # Define priors

    alpha_prior = tf.reduce_sum(tfp.distributions.Chi2(df=1, validate_args=False, allow_nan_stats=True).log_prob(alpha))
    beta1_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.203525, tf.float32), scale=tf.cast(0.298182, tf.float32)).log_prob(beta1))
    beta2_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.290288, tf.float32), scale=tf.cast(0.344120, tf.float32)).log_prob(beta2))
    beta3_prior = tf.reduce_sum(tfp.distributions.Normal(loc=tf.cast(0.258918, tf.float32), scale=tf.cast(0.338090, tf.float32)).log_prob(beta3))
    sigma_prior = tf.reduce_sum(tfp.distributions.Uniform(low=tf.cast(0.0, tf.float32), high=tf.cast(1000.0, tf.float32)).log_prob(sigma))

    mu = alpha + beta1_prior * Obama + beta2_prior  * Econ + beta3_prior  * Palistine

    _density_li = tfd.Normal(loc=mu, scale=sigma).log_prob(GooglePlus)
    density_li = tf.reduce_sum(_density_li, axis=0)

    # Sum of log probabilities
    return density_li + alpha_prior + beta1_prior + beta2_prior + beta3_prior + sigma_prior

In [268]:
# Define bijectors for transformations
bijectors = [
    tfp.bijectors.Square(),
    tfp.bijectors.Identity(),
    tfp.bijectors.Identity(),
    tfp.bijectors.Identity(),
    tfp.bijectors.SoftClip(low=0.0,high=1000.0)
]

# Initial state for the sampler
initial_state = [
    tf.zeros([], dtype=tf.float32, name='init_alpha'),
    tf.zeros([], dtype=tf.float32, name='init_beta1'),
    tf.zeros([], dtype=tf.float32, name='init_beta2'),
    tf.zeros([], dtype=tf.float32, name='init_beta3'),
    tf.ones([], dtype=tf.float32, name='init_sigma')
]

In [269]:
@tf.function()
def sample_chain(initial_state):
    theta = tfp.mcmc.sample_chain(
        #num_results=Nsteps,
        num_results=2000,
        num_burnin_steps=N_burn_in,
        # num_burnin_steps=1000,
        current_state=initial_state,
        kernel=tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.TransformedTransitionKernel(

                # NUTS instead of HMC
                inner_kernel=tfp.mcmc.NoUTurnSampler(
                    target_log_prob_fn=joint_log_prob,
                    max_tree_depth=8,
                    max_energy_diff=1000,

                    step_size=tf.cast(0.01, tf.float32)),    # STEPSIZE TUNE

                bijector= bijectors ),

            num_adaptation_steps=600),
        trace_fn=None)
    return theta

In [270]:
alpha = chisquare(1)
beta1 = normal(0.203525,0.298182)
beta2 = normal(0.290288,0.344120)
beta3 = normal(0.258918,0.338090)
sigma = uniform(0.0, 1000.0)

theta = sample_chain([alpha, beta1, beta2, beta3, sigma])
print(theta)

[<tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.04010506, 0.05499638, 0.09655105, ..., 0.75086415, 0.75086415,
       0.75086415], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([ 0.24527042,  0.3659576 ,  0.36984503, ..., -0.17178982,
       -0.17178982, -0.17178982], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.49989626, 0.4705842 , 0.64365506, ..., 0.51905715, 0.51905715,
       0.51905715], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([0.6520416 , 0.65453035, 0.57247823, ..., 0.04604121, 0.04604121,
       0.04604121], dtype=float32)>, <tf.Tensor: shape=(2000,), dtype=float32, numpy=
array([87.948425, 87.4834  , 86.93628 , ..., 16.32965 , 16.32965 ,
       16.32965 ], dtype=float32)>]


In [271]:
# Extract samples
posterior = [theta[0].numpy(), theta[1].numpy(), theta[2].numpy(), theta[3].numpy(), theta[4].numpy()]

# Create a DataFrame
posterior_df = pd.DataFrame(posterior).T  # Transpose the DataFrame

# Set column names
posterior_df.columns = ['alpha', 'beta1', 'beta2', 'beta3', 'sigma']
print(posterior_df)

         alpha     beta1     beta2     beta3      sigma
0     0.040105  0.245270  0.499896  0.652042  87.948425
1     0.054996  0.365958  0.470584  0.654530  87.483398
2     0.096551  0.369845  0.643655  0.572478  86.936279
3     0.096551  0.369845  0.643655  0.572478  86.936279
4     0.165752  0.320993  0.644690  0.529797  86.423889
...        ...       ...       ...       ...        ...
1995  0.750864 -0.171790  0.519057  0.046041  16.329651
1996  0.750864 -0.171790  0.519057  0.046041  16.329651
1997  0.750864 -0.171790  0.519057  0.046041  16.329651
1998  0.750864 -0.171790  0.519057  0.046041  16.329651
1999  0.750864 -0.171790  0.519057  0.046041  16.329651

[2000 rows x 5 columns]
